In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [43]:
class Conv_2d:
    def __init__(self, kernel_size, num_filters, stride = 1, pad = 2):
        self.filter_size = kernel_size
        self.num_filters = num_filters
        self.stride = stride
        self.pad = pad
        
    def zero_pad(self, original_inputs, pad):
        padded_inputs = np.pad(original_inputs, ((0, 0), (pad, pad), (pad, pad), (0, 0)), "constant", constant_values = 0)
        return padded_inputs
    
    def forward_single_step(self, screened_area, weight, bias):
        Z = np.sum(screened_area * weight) + bias
        return Z
    
    def forward_propagation(self, inputs):
        self.inputs = inputs
        n, n_H_inputs, n_W_inputs, n_C_inputs = self.inputs.shape
        self.Theta = np.random.randn(self.filter_size, self.filter_size, n_C_inputs, self.num_filters)
        self.theta = np.random.randn(1, 1, 1, self.num_filters)
        inputs_padding = self.zero_pad(self.inputs, self.pad)
        
        n_H = int((n_H_inputs + 2 * self.pad - self.filter_size) * 1.0 / self.stride + 1)
        n_W = int((n_W_inputs + 2 * self.pad - self.filter_size) * 1.0 / self.stride + 1)
        self.Z = np.zeros((n, n_H, n_W, self.num_filters))
        
        for i in range(n):
            for h in range(n_H):
                for w in range(n_W):
                    for c in range(self.num_filters):
                        h_start = h * self.stride
                        h_end = h * self.stride + self.filter_size
                        w_start = w * self.stride
                        w_end = w * self.stride + self.filter_size
                        screened_area = inputs_padding[i, h_start: h_end, w_start: w_end]
                        self.Z[i, h, w, c] = self.forward_single_step(screened_area, self.Theta[:, :, :, c], self.theta[:, :, :, c])
    
    def backward_propagation(self, dZ):
        

In [48]:
class Pool_2d:
    
    def __init__(self, kernel_size, stride = 1, pooling_mode = "max"):
        self.filter_size = kernel_size
        self.stride = stride
        assert pooling_mode in ["max", "mean"], "Invalid pooling mode"
        self.pooling_mode = pooling_mode
        
    def forward_propagation(self, inputs):
        self.inputs = inputs
        n, n_H_inputs, n_W_inputs, n_C_inputs = self.inputs.shape
        
        n_H = int((n_H_inputs - self.filter_size) * 1.0 / self.stride + 1)
        n_W = int((n_W_inputs - self.filter_size) * 1.0 / self.stride + 1)
        n_C = n_C_inputs
        
        self.Z = np.zeros((n, n_H, n_W, n_C))
        for i in range(n):
            for h in range(n_H):
                for w in range(n_W):
                    for c in range(n_C):
                        h_start = h * self.stride
                        h_end = h * self.stride + self.filter_size
                        w_start = w * self.stride
                        w_end = w * self.stride + self.filter_size
                        screened_area = self.inputs[i, h_start: h_end, w_start: w_end, c]
                        if self.pooling_mode == "max":
                            self.Z[i, h, w, c] = np.max(screened_area)
                        else:
                            self.Z[i, h, w, c] = np.mean(screened_area)
        return self.Z